In [35]:
from matplotlib import colors as mcolors
def rgb_to_hex(rgb: list) -> str:
    return mcolors.to_hex([x/255 for x in rgb])
rgb_to_hex([255, 0, 0])

'#ff0000'

In [26]:
import random

def generate_bright_color(hue: float = None) -> list:
    if hue is None:
        hue = random.random()
    
    # Use high saturation and value for brightness
    saturation = random.uniform(0.6, 1.0)
    value = random.uniform(0.9, 1.0)
    
    # Convert HSV to RGB
    h = hue * 6
    c = value * saturation
    x = c * (1 - abs(h % 2 - 1))
    m = value - c
    
    if h < 1:
        r, g, b = c, x, 0
    elif h < 2:
        r, g, b = x, c, 0
    elif h < 3:
        r, g, b = 0, c, x
    elif h < 4:
        r, g, b = 0, x, c
    elif h < 5:
        r, g, b = x, 0, c
    else:
        r, g, b = c, 0, x
    
    # Scale and convert to integer values
    return [int((r + m) * 255), int((g + m) * 255), int((b + m) * 255)]

# Example usage
print(generate_bright_color())

[236, 199, 13]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Tuple

def visualize_colors(colors: List[Tuple[int, int, int]], grid_size: Tuple[int, int] = (10, 10)):
    rows, cols = grid_size
    fig, axs = plt.subplots(rows, cols, figsize=(cols, rows))
    fig.subplots_adjust(hspace=0.1, wspace=0.1)

    for ax, color in zip(axs.flat, colors):
        ax.add_patch(plt.Rectangle((0, 0), 1, 1, facecolor=[x/255 for x in color]))
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

# Generate colors
num_colors = 100
colors = [generate_bright_color(i/num_colors) for i in range(num_colors)]

# Visualize colors
visualize_colors(colors)

In [39]:
import supervisely as sly
import os

project_id = 41774

api = sly.Api()

meta = api.project.get_meta(project_id)

{"message": "Workflow is disabled in development mode. To enable it, use `api.app.workflow.enable()` right after Api initialization.", "timestamp": "2024-09-23T15:56:28.442Z", "level": "warn"}


In [40]:
for cls in meta['classes']:
    if '_polygon' in cls['title']:
        meta['classes'].remove(cls)
meta['classes']

[{'id': 9372355,
  'title': 'airplane',
  'description': '',
  'shape': 'rectangle',
  'hotkey': '',
  'color': '#F20F0F'},
 {'id': 9379895,
  'title': 'apple',
  'description': '',
  'shape': 'rectangle',
  'hotkey': '',
  'color': '#FC584A'},
 {'id': 9372811,
  'title': 'backpack',
  'description': '',
  'shape': 'rectangle',
  'hotkey': '',
  'color': '#FB3B19'},
 {'id': 9379648,
  'title': 'banana',
  'description': '',
  'shape': 'rectangle',
  'hotkey': '',
  'color': '#F2633A'},
 {'id': 9379166,
  'title': 'baseball bat',
  'description': '',
  'shape': 'rectangle',
  'hotkey': '',
  'color': '#F57B47'},
 {'id': 9378931,
  'title': 'baseball glove',
  'description': '',
  'shape': 'rectangle',
  'hotkey': '',
  'color': '#FD8D49'},
 {'id': 9381190,
  'title': 'bear',
  'description': '',
  'shape': 'rectangle',
  'hotkey': '',
  'color': '#F2852C'},
 {'id': 9372673,
  'title': 'bench',
  'description': '',
  'shape': 'rectangle',
  'hotkey': '',
  'color': '#ED9635'},
 {'id': 93

In [42]:
api.project.update_meta(project_id, meta)

In [ ]:
from tqdm import tqdm
num_colors = 80
i = 0
for cls in tqdm(meta['classes']):
    if "_polygon" not in cls['title']:
        cls['title'] = cls['title'].replace("_rectangle", "")
        cls['color'] = rgb_to_hex(generate_bright_color(i/num_colors)).upper()
        i += 1
        assert cls['shape'] == 'rectangle'
        r = api.post("advanced.object_classes.editInfo", cls)
        assert r.status_code == 200, r.text